In [7]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the iexfinance api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [9]:
from iexfinance.stocks import get_historical_data
import iexfinance as iex

# Data Collection

In this step, you will need to use the IEX api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [10]:
#Set the List of Tickers
tickers = ["SPY", "AGG"]

In [21]:
# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-359)

In [22]:
# Get 1 year's worth of historical data
get_historical_data(tickers, start_date, end_date, output_format = 'pandas')

IEXQueryError: An error occurred while making the query.

In [ ]:
#Drop Extraneous Columns
df.drop(columns=['open', 'high', 'low', 'volume'], level=1, inplace=True)
df.head()

# Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:
1. Calculate the daily returns for the SPY and AGG closing prices.
2. Calculate volatility for both the SPY and AGG closing prices.
3. Find the last day's closing price for both stocks and save those as variables.
4. Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

### HINTS:
There are 252 trading days per year, so the number of records to generate for each Monte Carlo run will be 252 days * 30 years

In [ ]:
# Calculate the Daily ROI for the Stocks
daily_returns = df.pct_change()
daily_returns.head()

In [ ]:
# Calculate Value of Average Daily Returns
avg_daily_returns_spy = daily_returns.mean()['SPY']['close']
avg_daily_returns_agg = daily_returns.mean()['AGG']['close']

In [ ]:
# Calculate volatility
std_dev_daily_returns_spy = daily_returns.std()['SPY']['close']
std_dev_daily_returns_agg = daily_returns.std()['AGG']['close']

In [ ]:
# Save the last day's closing price
spy_last_price = df['SPY']['close'][-1]
agg_last_price = df['AGG']['close'][-1]

In [ ]:
# Setup the Monte Carlo Parameters
number_simulations = 100
number_records = 252
monte_carlo = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()

In [ ]:
# Run the Monte Carlo Simulation
for x in range (number_simulations):
    simulated_spy_prices = [spy_last_price]
    simulated_agg_prices = [agg_last_price]
    for y in range (number_records):
        simulated_spy_price = simulated_spy_prices[-1] * (1 + np.random.normal(avg_daily_returns_spy, std_dev_daily_returns_spy))
        simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_returns_agg, std_dev_daily_returns_agg))
        simulated_spy_prices.append(simulated_spy_price)
        simulated_agg_prices.append(simulated_agg_price)
    monte_carlo['SPY prices'] = pd.Series(simulated_spy_prices)
    monte_carlo['AGG prices'] = pd.Series(simulated_agg_prices)
    simulated_daily_returns = monte_carlo.pct_change()
    weights = [0.6, 0.4]
    portfolio_daily_returns = simulated_daily_returns.dot(weights)
    portfolio_cumulative_returns[x] = (1 + portfolio_daily_returns.fillna(0)).cumprod()
portfolio_cumulative_returns.head()

In [ ]:
# Visualize the Simulation
plot_title = f"{number_simulations} Simulations of Cumulative Portfolio Return Trajectories Over the Next {number_records} Trading Days"
portfolio_cumulative_returns.plot(legend=None, title=plot_title, figsize=(15,10))

In [ ]:
# Setup the Monte Carlo Parameters (~ 30 years)
number_simulations = 500
number_records = 252 * 30

In [ ]:
for x in range (number_simulations):
    simulated_spy_prices = [spy_last_price]
    simulated_agg_prices = [agg_last_price]
    for y in range (number_records):
        simulated_spy_price = simulated_spy_prices[-1] * (1 + np.random.normal(avg_daily_returns_spy, std_dev_daily_returns_spy))
        simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_returns_agg, std_dev_daily_returns_agg))
        simulated_spy_prices.append(simulated_spy_price)
        simulated_agg_prices.append(simulated_agg_price)
    monte_carlo['SPY prices'] = pd.Series(simulated_spy_prices)
    monte_carlo['AGG prices'] = pd.Series(simulated_agg_prices)
    simulated_daily_returns = monte_carlo.pct_change()
    weights = [0.6, 0.4]
    portfolio_daily_returns = simulated_daily_returns.dot(weights)
    portfolio_cumulative_returns[x] = (1 + portfolio_daily_returns.fillna(0)).cumprod()
portfolio_cumulative_returns.head()

In [ ]:
# Visualize the Simulation
plot_title = f"{number_simulations} Simulations of Cumulative Portfolio Return Trajectories Over the Next {number_records} Trading Days (~30 years)"
portfolio_cumulative_returns.plot(legend=None, title=plot_title, figsize=(15,10))

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
ending_cumulative_returns = portfolio_cumulative_returns.iloc[-1, :]
ending_cumulative_returns.head()

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 20 years)


In [ ]:
# Display the 90% confidence interval for the ending returns
confidence_interval = ending_culumative_returns.quantile(q[0.1, 0.9])
ending_cumulative_returns.plot(kind='hist', density=True, bins=10, figsize=(15,10))
plt.axvline(confidence_interval.iloc[0], color='r')
plt.axvline(confidence_interval.iloc[1], color='r')

In [ ]:
# Visualize the distribution of the ending returns
distribution = ending_cumulative_returns.plot(legend=None, title=plot_title, figsize=(15,10))

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# Set Initial Investment
initial_investment = 20000

In [ ]:
# New Confidence Interval
confidence_interval = ending_culumative_returns.quantile(q[0.1, 0.5, 0.9])

In [ ]:
# Calculate Investment Profit/loss
tenth_percentile = initial_investment + (initial_investment * new_confidence_interval.iloc[0])
fiftieth_percentile = initial_investment + (initial_investment * new_confidence_interval.iloc[1])
ninetieth_percentile = initial_investment + (initial_investment * new_confidence_interval.iloc[2])

In [ ]:
# Print the Results
print(f"Expected cumulative returns at 30 years: 10th percentile => {tenth_percentile} 50th percentile => {fiftieth_percentile} 90th percentile => {ninetieth_percentile}
")

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
# Set Initial Investment
new_initial = 20000 + (20000*.5)

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 